In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [16]:
cancer_data = pd.read_csv("C:\\Users\\Polar\\Documents\\ESCUELA\\5TO_SEMESTRE\\MAKINITAS\\practica5\\breast-cancer.csv")
cancer_data.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [17]:
# Separate input features and output feature
X = cancer_data.drop(columns = ['Class'])       # input features
Y = cancer_data.Class     # output feature

In [18]:
cat_cols = X.select_dtypes(include = 'object').columns 
cancer_data = pd.get_dummies(cancer_data, columns=cat_cols)
cancer_data.head()

,Class,deg-malig,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,menopause_ge40,menopause_lt40,...,breast_left,breast_right,breast-quad_?,breast-quad_central,breast-quad_left_low,breast-quad_left_up,breast-quad_right_low,breast-quad_right_up,irradiat_no,irradiat_yes
0,no-recurrence-events,3,False,True,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,True,False
1,no-recurrence-events,2,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,True,True,False
2,no-recurrence-events,2,False,False,True,False,False,False,False,False,...,True,False,False,False,True,False,False,False,True,False
3,no-recurrence-events,2,False,False,False,False,True,False,True,False,...,False,True,False,False,False,True,False,False,True,False
4,no-recurrence-events,2,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,True,False


In [19]:
# Separate input features and output feature
X = cancer_data.drop(columns = ['Class'])       # input features
Y = cancer_data.Class     # output feature

In [20]:
from sklearn.preprocessing import StandardScaler
scaled_features = StandardScaler().fit_transform(X)
scaled_features = pd.DataFrame(data=scaled_features)
scaled_features.columns= X.columns
# Let's see how the data looks after scaling
scaled_features.head()

,deg-malig,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,menopause_ge40,menopause_lt40,menopause_premeno,...,breast_left,breast_right,breast-quad_?,breast-quad_central,breast-quad_left_low,breast-quad_left_up,breast-quad_right_low,breast-quad_right_up,irradiat_no,irradiat_yes
0,1.290564,-0.059235,2.635231,-0.677631,-0.710819,-0.498907,-0.146385,-0.906452,-0.158397,0.95219,...,0.938924,-0.938924,-0.059235,-0.281505,1.264911,-0.716399,-0.302660,-0.361158,0.558504,-0.558504
1,-0.066426,-0.059235,-0.379473,1.475730,-0.710819,-0.498907,-0.146385,-0.906452,-0.158397,0.95219,...,-1.065049,1.065049,-0.059235,-0.281505,-0.790569,-0.716399,-0.302660,2.768875,0.558504,-0.558504
2,-0.066426,-0.059235,-0.379473,1.475730,-0.710819,-0.498907,-0.146385,-0.906452,-0.158397,0.95219,...,0.938924,-0.938924,-0.059235,-0.281505,1.264911,-0.716399,-0.302660,-0.361158,0.558504,-0.558504
3,-0.066426,-0.059235,-0.379473,-0.677631,-0.710819,2.004381,-0.146385,1.103202,-0.158397,-1.05021,...,-1.065049,1.065049,-0.059235,-0.281505,-0.790569,1.395870,-0.302660,-0.361158,0.558504,-0.558504
4,-0.066426,-0.059235,-0.379473,1.475730,-0.710819,-0.498907,-0.146385,-0.906452,-0.158397,0.95219,...,-1.065049,1.065049,-0.059235,-0.281505,-0.790569,-0.716399,3.304038,-0.361158,0.558504,-0.558504


In [25]:
from sklearn.preprocessing import LabelEncoder
# Convertir la variable de salida categórica a valores numéricos
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

# Definir el método de validación cruzada Leave-One-Out
cv = LeaveOneOut()

# Construir el modelo de regresión lineal
model = LinearRegression()

# Usar LOOCV para evaluar el modelo
scores = cross_val_score(model, scaled_features, Y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)

# Ver el RMSE (Root Mean Squared Error)
rmse = sqrt(mean(absolute(scores)))
print("RMSE: ", rmse)

RMSE:  6309355561611.731
